In [ ]:
pip install rasterio pandas numpy matplotlib

In [1]:
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# === 1. Caminhos para os arquivos GeoTIFF ===
path_satelite = 'indices_completos_30m_corrigido.tif'
path_dems = 'novo_empilhadoDEM.tif'
output_stack = 'raster_empilhado.tif'

In [3]:
# === 2. Abrir os rasters com rasterio ===
with rasterio.open(path_satelite) as s:
    sat_data = s.read().astype('float32')
    nodata_s = s.nodata
    profile = s.profile.copy() 
    if nodata_s is not None:
        sat_data[sat_data == nodata_s] = np.nan

with rasterio.open(path_dems) as src_d:
    dem_data = src_d.read().astype('float32')
    
    # Detectar ou forçar valor nodata
    nodata_dem = src_d.nodata if src_d.nodata is not None else -999999.0
    
    # Substituir nodata por NaN
    dem_data[dem_data == nodata_dem] = np.nan
    dem_profile = src_d.profile

In [4]:
# === 3. Verificar dimensões (precisam ser iguais) ===
if sat_data.shape[1:] != dem_data.shape[1:]:
    raise ValueError("Os rasters não têm as mesmas dimensões espaciais")
else :
    print("Dimensões dos rasters são compatíveis.")

if s.crs != src_d.crs or s.transform != src_d.transform:
    raise ValueError("Os rasters têm projeções ou grids diferentes")


Dimensões dos rasters são compatíveis.


In [5]:
# === Remover a banda Dist_Rios do dem_data ===
# (Assumindo que 'Dist_Rios' é a 2ª banda, com índice 1, do raster 'novo_empilhadoDEM.tif')

print(f"Bandas DEM antes de remover Dist_Rios: {dem_data.shape[0]}")
# Remove a segunda banda (índice 1) do eixo 0 (eixo das bandas)
dem_data = np.delete(dem_data, 1, axis=0) 
print(f"Bandas DEM após remover Dist_Rios: {dem_data.shape[0]}")

Bandas DEM antes de remover Dist_Rios: 10
Bandas DEM após remover Dist_Rios: 9


In [6]:
# ===  Remover a banda EVI ===
print("Bandas espectrais antes:", sat_data.shape[0])
sat_data = np.delete(sat_data, 5, axis=0)
print("Bandas espectrais após remover EVI:", sat_data.shape[0])

Bandas espectrais antes: 7
Bandas espectrais após remover EVI: 6


In [7]:
# 4. Empilhar as bandas espectrais e DEMs ===
full_stack = np.concatenate([sat_data, dem_data], axis=0)  # shape: (total_bands, rows, cols)

In [8]:
# 5. Transformar raster empilhado em tabela (pixels x bandas) //cada pixel vira uma linha e cada banda uma coluna.
rows, cols = full_stack.shape[1:]
n_bands = full_stack.shape[0]
flat_stack = full_stack.reshape(n_bands, -1).T  # shape: (n_pixels, n_bands)

In [9]:
# ===  Salvar o raster empilhado ===
profile = s.profile
profile.update(count=full_stack.shape[0], dtype='float32')

with rasterio.open(output_stack, 'w', **profile) as dst:
    dst.write(full_stack)

print(f"Raster empilhado salvo em: {output_stack}")

Raster empilhado salvo em: raster_empilhado.tif


In [10]:
# === 6. Converter para DataFrame e remover pixels inválidos ===
df = pd.DataFrame(flat_stack)
df_clean = df.replace([-np.inf, np.inf], np.nan).dropna().reset_index(drop=True)

In [11]:
# === 8. Nomear as colunas ===
colunas = [
    # Índices espectrais
    'NDVI', 'NDWI', 'NDBI', 'MNDWI', 'SAVI', 'BSI',

    # Variáveis topográficas e hidrológicas (ordem real no raster)
    'TWI',
    'Altitude', 'Curvatura_H', 'Curvatura_V', 'Declividade',
    'Divisores', 'Forma_terreno', 'Orientacao', 'Relevo'
]

df_clean.columns = colunas[:len(df_clean.columns)]



In [12]:
# Remover EVI 
if 'EVI' in df_clean.columns:
    df_clean = df_clean.drop(columns=['EVI'])

In [13]:
print("Dimensão final da matriz:", df_clean.shape)
print(df_clean.head())

Dimensão final da matriz: (675192, 15)
       NDVI      NDWI      NDBI     MNDWI      SAVI       BSI        TWI  \
0 -0.431217  0.625762 -0.457627  0.842209 -0.645971 -0.186107  20.203773   
1 -0.408740  0.604131 -0.474359  0.838264 -0.612323 -0.183938  20.201118   
2 -0.439791  0.630397 -0.441077  0.838364 -0.658824 -0.181642  20.097979   
3 -0.445910  0.634783 -0.438356  0.839530 -0.667984 -0.183938  19.961836   
4 -0.421400  0.622088 -0.450331  0.837732 -0.631266 -0.193506  20.335741   

     Altitude  Curvatura_H  Curvatura_V  Declividade  Divisores  \
0  219.298004     1.092186     0.006342    15.907906       17.0   
1  218.755997     0.809141     0.006826    12.912099       15.0   
2  217.466003     0.657675    -0.002509    10.241418       11.0   
3  215.852005    -0.224559    -0.014189    13.280601        4.0   
4  219.792007     1.965719     0.020046    10.290605       18.0   

   Forma_terreno  Orientacao    Relevo  
0            6.0  279.364441  0.916624  
1            7.0   

In [14]:
# Salvar como CSV
df_clean.to_csv("dados_para_modelagem.csv", index=False)